In [ ]:
!sudo pip install -e /workspace/src/aymurai tensorflow_hub tensorflow_text

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
import flair
import logging


flair.logger.setLevel(logging.ERROR)

In [ ]:
from aymurai.spacy.display import DocRender
from aymurai.datasets.ar_juz_pcyf_10.annotations import (
    ArgentinaJuzgadoPCyF10LabelStudioAnnotations,
)

render = DocRender()
docs = ArgentinaJuzgadoPCyF10LabelStudioAnnotations(
    "/resources/data/restricted/annotations/20221130-bis/"
).data
# docs = ArgentinaJuzgadoPCyF10DocsDataset().data
sample = docs[:10]

print(len(docs))

In [ ]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU' or x.device_type == 'CPU']

get_available_devices()

## Pipeline

In [ ]:
from aymurai.pipeline import AymurAIPipeline
from aymurai.models.flair.core import FlairModel
from aymurai.models.flair.utils import FlairTextNormalize
from aymurai.transforms.entity_subcategories.regex import RegexSubcategorizer
from aymurai.transforms.entity_subcategories.usem import USEMSubcategorizer

config = {
    "preprocess": [
        (FlairTextNormalize, {}),
    ],
    "models": [
        (
            FlairModel,
            {
                "basepath": "/resources/pipelines/examples/flair-simple/FlairModel",
                "split_doc": True,
                "device": "cpu",
            },
        )
    ],
    "postprocess": [
        (RegexSubcategorizer, {}),
        (
            USEMSubcategorizer,
                {
                    "category": "CONDUCTA",
                    "subcategories_path": '/resources/pipelines/examples/flair-simple/UsemEmbeddings/conduct_options.txt',
                    "response_embeddings_path": "/resources/pipelines/examples/flair-simple/UsemEmbeddings/conduct_embeddings.npy",
                    "device": '/cpu:0'
                }
        ),
        (
            USEMSubcategorizer,
                {
                    "category": "CONDUCTA_DESCRIPCION",
                    "subcategories_path": '/resources/pipelines/examples/flair-simple/UsemEmbeddings/conduct_desc_options.txt',
                    "response_embeddings_path": "/resources/pipelines/examples/flair-simple/UsemEmbeddings/conduct_descr_embeddings.npy",
                    "device": '/cpu:0'
                }
        ),
        (
            USEMSubcategorizer,
                {
                    "category": "DETALLE",
                    "subcategories_path": '/resources/pipelines/examples/flair-simple/UsemEmbeddings/detail_options.txt',
                    "response_embeddings_path": "/resources/pipelines/examples/flair-simple/UsemEmbeddings/detail_embeddings.npy",
                    "device": '/cpu:0'
                }
        ),
        (
            USEMSubcategorizer,
                {
                    "category": "OBJETO_DE_LA_RESOLUCION",
                    "subcategories_path": '/resources/pipelines/examples/flair-simple/UsemEmbeddings/object_options.txt',
                    "response_embeddings_path": "/resources/pipelines/examples/flair-simple/UsemEmbeddings/object_embeddings.npy",
                    "device": '/cpu:0'
                }
        ),
    ],
    # "multiprocessing": {},
    "use_cache": False,
    # 'log_level': 'debug'
}

In [ ]:
pipeline = AymurAIPipeline(config)
results = pipeline.preprocess(sample[:1])
results = pipeline.predict(results)


In [ ]:

postprocessed = pipeline.postprocess(results)

In [ ]:
idx = -1

render(postprocessed[idx])